In [ ]:
import torch
import torch.nn as nn
# 기본값 : torch.nn.Conv2d(in_channesls, out_channels, kernel_size, stride=1, padding=0, bias=True)
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=11, stride=4,padding=0)
conv

In [ ]:
inputs = torch.Tensor(1,1,227,227)
outputs = conv(inputs)
print(outputs)
outputs.shape

In [ ]:
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=7,stride=2, padding=0)
print(conv)
inputs = torch.Tensor(1,1,64,64)
outputs = conv(inputs)
print(outputs)
print('outputs shape:',outputs.shape)

In [ ]:
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5,stride=1, padding=2)
print(conv)
inputs = torch.Tensor(1,1,32,632)
outputs = conv(inputs)
print(outputs)
print('outputs shape:',outputs.shape)

In [ ]:
# 예제 4
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5,stride=1, padding=0)
print(conv)
inputs = torch.Tensor(1,1,32,64)
outputs = conv(inputs)
print(outputs)
print('outputs shape:',outputs.shape)

In [ ]:
# 예제 5
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3,stride=1, padding=1)
print(conv)
inputs = torch.Tensor(1,1,64,32)
outputs = conv(inputs)
print(outputs)
print('outputs shape:',outputs.shape)

In [ ]:
# torch.nn.MaxPool2d(kernel_size)

import torch
import torch.nn as nn
inputs = torch.Tensor(1,1,28,28)
conv1 = nn.Conv2d(in_channels=1, out_channels=5,kernel_size=5)
pool = nn.MaxPool2d(2)
out = conv1(inputs)
out2 = pool(out)
print(out.size())
print(out2.size())

# Lab-10-2 mnist cnn
1. 라이브러리를 가져온다 (torch, torchvision, matplotlib)
2. GPU 사용 설정하고 random value를 위한 seed 설정
3. 학습에 사용되는 parameter 설정 (learning_rate, training_epochs, batch_size, etc)
4. 데이터셋을 가져오고 (학습에 쓰기 편하기) loader 만들기
5. 학습 모델 만들기 (`class CNN(torch.nn.Module)`)
6. Loss function (Criterion)을 선택하고 최적화 도구 선택(Optimizer)
7. 모델 학습 및 Loss check(Criterion의 Output)
8. 학습된 모델의 성능을 확인한다.

## 우리가 만들 CNN 구조
- 데이터 -> Layer1(Conv->ReLU->MaxPool) -> Layer2(Conv->ReLU->MaxPool) -> view -> FC -> Cross Entropy Loss(SoftMax -> NLL loss)
- (Layer1) Convolution layer = (in_c=1, out_c=32, kernel_size=3, stride=1, padding=1)
- (Layer1) MaxPool layer = (kernel_size=2, stride=2)

- (Layer2) Convolution layer = (in_c=32, out_c=64, kernel_size=3, stride=1, padding=1)
- (Layer2) MaxPool layer = (kernel_size=2, stride=2)

- view => (batch_size x [7,7,64] => batch_size x [3136])
- Fully_Conneted layer => (input=3136, output=10)

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda'

torch.manual_seed(777)
torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
# CNN model
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape = (?, 28, 28, 1)
        # conv -> (?, 28, 28, 32)
        # Pool -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # L2 ImgIn shape = (?, 14, 14, 32)
        # Conv -> (?, 14, 14, 64)
        # Pool -> (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Final FC 7*7*64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(in_features=7*7*64, out_features=10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) 
        # Flatten them for FC
        out = self.fc(out)
        
        return out

In [ ]:
# instantiate CNN model
model = CNN().to(device)

In [ ]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
# Softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

In [ ]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

## Lab-10-4-1 ImageFolder1
https://www.youtube.com/watch?v=KDVOAjaTnDU&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=22


In [ ]:
import torchvision
from torchvision import transforms

from torch.utils.data import dataloader

In [ ]:
from matplotlib.pyplot import imshow
%matplotlib inline # Notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것

In [ ]:
trans = transforms.Compose([
    transforms.Resize((64,128))
])

train_data = torchvision.datasets.ImageFolder(root="/home/hb/python/public-repo/deeplearningzerotoall/PyTorch/custom_data/origin_data", transform=trans)

`transforms` 함수는 이미지 형태를 변경해주는 것

In [ ]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)

    if(label == 0):
        data.save('/home/hb/python/public-repo/deeplearningzerotoall/PyTorch/custom_data/origin_data/gray/%d_%d.jpeg'%(num, label))
    else:
        data.save('/home/hb/python/public-repo/deeplearningzerotoall/PyTorch/custom_data/origin_data/red/%d_%d.jpeg'%(num,label))


## Lab-10-4-2 ImageFolder2
https://www.youtube.com/watch?v=NqwUmCZbjNA&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=23

- Lab-10-4-2에서는 `ImageFolder` 사용법 공부
- 오늘은 내 사진을 분류하는 딥러닝 모델 학습

#### 1. 라이브러리 로드

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

#### 2. GPU 사용 설정 및 random value를 위한 seed 설정

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

#### 3. 데이터셋을 가져오고 (학습에 쓰기 편하게) loader만들기 

In [19]:
trans = transforms.Compose([
    transforms.ToTensor()
    ])

train_data = torchvision.datasets.ImageFolder(root='/home/hb/python/public-repo/deeplearningzerotoall/PyTorch/custom_data/train_data', transform=trans)


In [20]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

#### 4. 학습 모델 만들기 (`class CNN(torch.nn.Module)`)

<img src="/home/hb/python/public-repo/image/스크린샷 2022-06-07 오후 9.57.03.png" width='600' height='400'/>



In [21]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out
        

#### 4.* testing
중간에 `print(out.shape)`를 넣고 아래 코드를 돌리면 layer3에 넣어줘야 될 fc의 형태를 알 수 있다.

torch.Size([3, 6, 30, 62])  
torch.Size([3, 16, 13, 29])  
torch.Size([3, 6032])  

In [22]:
#testing 
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

#### 5. Loss function (Criterion)을 선택하고, 최적화 도구 선택(optimizer)

In [23]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

#### 6. 모델 학습 및 Loss check (Criterion의 output)

In [24]:
total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')   

[Epoch:1] cost = 0.6551641225814819
[Epoch:2] cost = 0.49421167373657227
[Epoch:3] cost = 0.23495927453041077
[Epoch:4] cost = 0.06872614473104477
[Epoch:5] cost = 0.026471389457583427
[Epoch:6] cost = 0.013910708017647266
[Epoch:7] cost = 0.008653281256556511
Learning Finished!


#### 6-1. 학습한 모델 저장 후 불러오기

In [25]:
# import os
# os.mkdir("./model")

In [26]:
torch.save(net.state_dict(), "./model/model.pth")

In [27]:
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [28]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0915,  0.0031, -0.0173, -0.0214,  0.0929], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([-0.0915,  0.0031, -0.0173, -0.0214,  0.0929], device='cuda:0',
       grad_fn=<SelectBackward0>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

#### 7. 학습시킨 데이터를 테스트하기

In [29]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])

test_data = torchvision.datasets.ImageFolder(root='/home/hb/python/public-repo/deeplearningzerotoall/PyTorch/custom_data/test_data', transform=trans
)

In [30]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [31]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
